In [1]:
import pandas as pd 
import requests
import time

def get_tvl_data(coin_id):
    """
    This function fetches TVL data from Defillama and returns it as a DataFrame.
    
    Args:
        coin_id: the name of the crypto we are getting the data
    
    Returns:
        pd.DataFrame: Melted dataframe with Protocol, date (variable), and value columns
    """
    # Construct the URL
    full_url_all = f'https://api.llama.fi/simpleChainDataset/{coin_id}?pool2=true&staking=true&borrowed=true&doublecounted=true&liquidstaking=true&vesting=true&govtokens=true'
    full_url_borrows = f'https://api.llama.fi/simpleChainDataset/{coin_id}?pool2=true&staking=true&borrowed=false&doublecounted=true&liquidstaking=true&vesting=true&govtokens=true'

    # Download the CSV
    response = requests.get(full_url_all, headers={'User-agent': 'Price Scrapper'})
    response2 = requests.get(full_url_borrows, headers={'User-agent': 'Price Scrapper'})
    
    if response.status_code == 429:
        time.sleep(int(response.headers["Retry-After"]))
        # Retry the request
        response = requests.get(full_url_all, headers={'User-agent': 'Price Scrapper'})
        response2 = requests.get(full_url_borrows, headers={'User-agent': 'Price Scrapper'})
    
    if response.status_code == 200:
        # Read CSV directly from response content into DataFrame
        from io import StringIO
        tvl = pd.read_csv(StringIO(response.content.decode('utf-8')))
        tvl2 = pd.read_csv(StringIO(response2.content.decode('utf-8')))
        
        # Melt the dataframe
        tvl = tvl.melt(id_vars='Protocol')
        tvl = tvl[tvl['Protocol']=='Folks Finance Lending'].reset_index(drop=True)

        tvl2 = tvl2.melt(id_vars='Protocol')
        tvl2 = tvl2[tvl2['Protocol']=='Folks Finance Lending'].reset_index(drop=True)
        

        # Convert date column
        tvl['variable'] = pd.to_datetime(tvl['variable'], format="%d/%m/%Y")
        tvl2['variable'] = pd.to_datetime(tvl2['variable'], format="%d/%m/%Y")
        

        # Optionally rename columns for clarity
        tvl = tvl.rename(columns={'variable': 'date', 'value': 'tvl'})
        tvl2 = tvl2.rename(columns={'variable': 'date', 'value': 'no_borrows'})
        
        tvl = tvl.merge(tvl2[['date','no_borrows']], on='date', how='left')
        tvl['borrows'] = tvl['tvl'] - tvl['no_borrows']
        return tvl
    else:
        print(f"Failed to download CSV for {coin_id}. Status code: {response.status_code}")
        return None

In [2]:
tvl = get_tvl_data('algorand')
tvl

,Protocol,date,tvl,no_borrows,borrows
0,Folks Finance Lending,2021-06-05,NaN,NaN,NaN
1,Folks Finance Lending,2021-06-06,NaN,NaN,NaN
2,Folks Finance Lending,2021-06-07,NaN,NaN,NaN
3,Folks Finance Lending,2021-06-08,NaN,NaN,NaN
4,Folks Finance Lending,2021-06-09,NaN,NaN,NaN
...,...,...,...,...,...
1688,Folks Finance Lending,2026-01-18,5.344434e+07,3.627318e+07,1.717116e+07
1689,Folks Finance Lending,2026-01-19,4.887700e+07,3.315299e+07,1.572401e+07
1690,Folks Finance Lending,2026-01-20,4.988225e+07,3.387711e+07,1.600514e+07
1691,Folks Finance Lending,2026-01-21,4.799719e+07,3.255584e+07,1.544136e+07


In [4]:
# Ensure date column is datetime
tvl['date'] = pd.to_datetime(tvl['date'])

# Get last 12 months
cutoff_date = tvl['date'].max() - pd.DateOffset(months=12)

borrows = tvl.loc[tvl['date'] >= cutoff_date, ['date', 'borrows']]
borrows

,date,borrows
1327,2025-01-22,4.963514e+07
1328,2025-01-23,5.101180e+07
1329,2025-01-24,5.053363e+07
1330,2025-01-25,4.843628e+07
1331,2025-01-26,5.027811e+07
...,...,...
1688,2026-01-18,1.717116e+07
1689,2026-01-19,1.572401e+07
1690,2026-01-20,1.600514e+07
1691,2026-01-21,1.544136e+07


In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from datetime import datetime, timedelta, timezone
import warnings
import clickhouse_connect
import os
import streamlit as st
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")


def get_client():
    client = clickhouse_connect.get_client(
            host=DB_HOST,
            port=DB_PORT,
            user=DB_USER,
            password=DB_PASS,
            secure=False
        )
    return client

def run_query(query: str):
    client = get_client()
    result = client.query(query)
    return result.result_rows, result.column_names



def date_to_unix_timestamp(start_date_str, end_date_str):
  """
  Converts start and end dates in YYYY-MM-DD format to Unix timestamps.

  Args:
    start_date_str: Start date string in YYYY-MM-DD format.
    end_date_str: End date string in YYYY-MM-DD format.

  Returns:
    A tuple containing the start and end Unix timestamps.
  """

  start_date = datetime.strptime(start_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
  end_date = datetime.strptime(end_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc) + timedelta(days=1) - timedelta(seconds=1) 

  start_timestamp = int(start_date.timestamp())
  end_timestamp = int(end_date.timestamp())

  return start_timestamp, end_timestamp

def get_close_price(start_date, end_date, asset_id):
    """
    Fetches an asset historical data from the API for the given interval.
    
    Args:
        start_timestamp: start unix timestamp
        end_timestamp: end unix timestamp

    Returns:
        pd.DataFrame: Dataframe containing the fetched data in daily intervals.
    """
    start_unix, end_unix = date_to_unix_timestamp(start_date, end_date)

    price_feed = f'https://indexer.vestige.fi/assets/{asset_id}/candles?network_id=0&interval=86400&start={start_unix}&end={end_unix}&denominating_asset_id=0&volume_in_denominating_asset=false'

    response = requests.get(price_feed)
    data = response.json()
    df = pd.DataFrame(data)
    return df

def combined_mcap_df(mcap_df):
    """
    Calculates the total market cap in USD by fetching price data for each asset
    and multiplying by their circulating supply.
    
    Args:
        mcap_df: DataFrame with date column and supply columns for each asset
                 (format: 'supply_{asset_id}_{asset_name}')
    
    Returns:
        pd.DataFrame: DataFrame with date and total_mcap_usd columns
    """
    start_date = str(mcap_df['date'].iloc[0])
    end_date = str(mcap_df['date'].iloc[-1])
    
    # Extract asset IDs and names from column names
    asset_info = []
    for col in mcap_df.columns:
        if col not in ['date', 'Date']:
            parts = col.split('_')
            if len(parts) >= 3 and parts[0] == 'supply':
                asset_id = parts[1]
                asset_name = parts[2].lower()
                asset_info.append({'id': asset_id, 'name': asset_name, 'col': col})
    
    # Create a copy of the dataframe with date
    result_df = mcap_df[['date']].copy()
    result_df['date'] = pd.to_datetime(result_df['date'])
    
    # Initialize total mcap column
    result_df['total_mcap_usd'] = 0.0
    
    # Fetch price data for each asset and calculate mcap
    for asset in asset_info:
        try:
            # Get price data
            price_df = get_close_price(start_date, end_date, asset['id'])
            
            # Convert timestamp to date
            price_df['date'] = pd.to_datetime(price_df['timestamp'], unit='s').dt.date
            price_df['date'] = pd.to_datetime(price_df['date'])
            
            # Merge with mcap data
            merged = result_df.merge(
                price_df[['date', 'close']], 
                on='date', 
                how='left'
            )
            
            # Forward fill missing prices
            merged['close'] = merged['close'].ffill()
            
            # Calculate mcap for this asset (supply * price)
            asset_mcap = mcap_df[asset['col']].fillna(0).values * merged['close'].fillna(0).values
            
            # Add to total mcap
            result_df['total_mcap_usd'] += asset_mcap
            
            # Optionally store individual asset mcap
            result_df[f"{asset['name']}_mcap_usd"] = asset_mcap
            
        except Exception as e:
            print(f"Error processing asset {asset['name']} (ID: {asset['id']}): {e}")
            continue
    
    return result_df

def combined_volume_df(vol_df):
    """
    Calculates the total market cap in USD by fetching price data for each asset
    and multiplying by their circulating supply.
    
    Args:
        mcap_df: DataFrame with date column and supply columns for each asset
                 (format: 'supply_{asset_id}_{asset_name}')
    
    Returns:
        pd.DataFrame: DataFrame with date and total_mcap_usd columns
    """
    start_date = str(vol_df['date'].iloc[0])
    end_date = str(vol_df['date'].iloc[-1])
    
    # Extract asset IDs and names from column names
    asset_info = []
    for col in vol_df.columns:
        if col not in ['date', 'Date']:
            parts = col.split('_')
            if len(parts) >= 3 and parts[0] == 'volume':
                asset_id = parts[1]
                asset_name = parts[2].lower()
                asset_info.append({'id': asset_id, 'name': asset_name, 'col': col})
    
    # Create a copy of the dataframe with date
    result_df = vol_df[['date']].copy()
    result_df['date'] = pd.to_datetime(result_df['date'])
    
    # Initialize total mcap column
    result_df['total_vol_usd'] = 0.0
    
    # Fetch price data for each asset and calculate mcap
    for asset in asset_info:
        try:
            # Get price data
            price_df = get_close_price(start_date, end_date, asset['id'])
            
            # Convert timestamp to date
            price_df['date'] = pd.to_datetime(price_df['timestamp'], unit='s').dt.date
            price_df['date'] = pd.to_datetime(price_df['date'])
            
            # Merge with mcap data
            merged = result_df.merge(
                price_df[['date', 'close']], 
                on='date', 
                how='left'
            )
            
            # Forward fill missing prices
            merged['close'] = merged['close'].ffill()
            
            # Calculate mcap for this asset (supply * price)
            asset_vol = vol_df[asset['col']].fillna(0).values * merged['close'].fillna(0).values
            
            # Add to total mcap
            result_df['total_vol_usd'] += asset_vol
            
            # Optionally store individual asset mcap
            result_df[f"{asset['name']}_vol_usd"] = asset_vol
            
        except Exception as e:
            print(f"Error processing asset {asset['name']} (ID: {asset['id']}): {e}")
            continue
    
    return result_df

In [9]:
VOLUME = """
    SELECT
      dateTrunc('month', date) as date,
      SUM(volume_246516580_GOLD$) as volume_246516580_GOLD$,
      SUM(volume_246519683_SILVER$) as volume_246519683_SILVER$,
      SUM(volume_1241944285_Gold) as volume_1241944285_Gold
    FROM c_algorand.commodities_volume
    WHERE date BETWEEN today() - INTERVAL 12 MONTH AND today()
    group by date
    order by date
    """

rows, cols = run_query(VOLUME)
vol_df = pd.DataFrame(rows, columns=cols)
vol_df = combined_volume_df(vol_df)
last30d_vol = vol_df[-30:]['total_vol_usd'].sum()
vol_delta = last30d_vol/vol_df[-60:-30]['total_vol_usd'].sum() - 1

/var/folders/xz/5bn1vdj146n6586g9r465k_40000gn/T/ipykernel_47605/1311788855.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  vol_delta = last30d_vol/vol_df[-60:-30]['total_vol_usd'].sum() - 1


In [10]:
vol_df

,date,total_vol_usd,gold$_vol_usd,silver$_vol_usd,gold_vol_usd
0,2025-02-01,7.185487e+06,4.185340e+06,2.952156e+06,47991.133854
1,2025-03-01,5.484630e+06,3.051885e+06,2.368217e+06,64528.943377
2,2025-04-01,1.045414e+07,5.769706e+06,4.615986e+06,68447.842256
3,2025-05-01,4.018460e+06,2.281042e+06,1.662201e+06,75217.512338
4,2025-06-01,4.633210e+06,2.864860e+06,1.687766e+06,80584.142904
5,2025-07-01,4.422552e+06,2.451443e+06,1.747123e+06,223986.427166
6,2025-08-01,2.791663e+06,1.430276e+06,1.263049e+06,98338.493697
7,2025-09-01,4.073824e+06,1.850373e+06,2.074665e+06,148786.066013
8,2025-10-01,5.924783e+06,2.985086e+06,2.759379e+06,180318.872648
9,2025-11-01,4.336590e+06,2.154947e+06,2.029562e+06,152080.504496
